In [13]:
# 1. Instalação e imports
!pip install -q -U transformers accelerate bitsandbytes
import os
import subprocess
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
# 2. Clonando repositorio
if not os.path.exists('crawl4ai'):
    !git clone https://github.com/unclecode/crawl4ai.git --quiet

In [15]:
# 3. Extração de evidências
def get_deep_evidence():
    # Tags e Datas
    tags = subprocess.check_output(
        "cd crawl4ai && git tag --sort=-creatordate --format='%(refname:short) (%(creatordate:short))' | head -n 8", 
        shell=True).decode()
    
    # Workflows
    try:
        workflows = subprocess.check_output("ls crawl4ai/.github/workflows", shell=True).decode()
    except: workflows = "Não listado"

    # Branches
    all_branches = subprocess.check_output("cd crawl4ai && git branch -r", shell=True).decode()
    has_develop = "develop" in all_branches
    
    # Ler o CHANGELOG.MD
    try:
        changelog = subprocess.check_output("head -n 50 crawl4ai/CHANGELOG.md", shell=True).decode()
    except:
        changelog = "Arquivo CHANGELOG.md não encontrado no caminho especificado."

    return tags, has_develop, workflows, changelog

tags_info, dev_branch_found, workflows_info, changelog_info = get_deep_evidence()

In [16]:
# 4. Configuração do Modelo (Mistral-7B)
model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True
)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


In [17]:
# 5. Prompt
contexto_real = f"""
--- DADOS TÉCNICOS ---
HISTÓRICO DE TAGS:
{tags_info}

CONTEÚDO RECENTE DO CHANGELOG:
{changelog_info}

ESTRUTURA:
- Branch 'develop' detectada: {'Sim' if dev_branch_found else 'Não'}
- Workflows: {workflows_info}
"""
prompt = f"<s>[INST] Você é um Analista de Engenharia de Software Sênior. Analise as evidências do projeto Crawl4AI e forneça um veredito técnico completo em PORTUGUÊS.\n\nEVIDÊNCIAS:\n{contexto_real}\n\nPERGUNTAS:\n1. Identifique o Modelo de Branching (GitHub Flow ou Gitflow) e justifique.\n2. Identifique a Estratégia de Release (Rapid Release, LTS ou Release Train) e justifique com base nas datas e no Changelog.\n3. Detalhe o fluxo de trabalho observado.\n4. Explique as regras de governança do projeto. [/INST]"

In [18]:
# 6. Execução
print("\n" + "="*50)
print("VEREDITO TÉCNICO FINAL:")
print("="*50 + "\n")

# Gerando a resposta
output = generator(prompt, max_new_tokens=1200, do_sample=False)

full_text = output[0]['generated_text']

if "[/INST]" in full_text:
    analysis = full_text.split("[/INST]")[-1].strip()
else:
    analysis = full_text

print(analysis)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



VEREDITO TÉCNICO FINAL:

1. O modelo de branching utilizado no projeto Crawl4AI é o GitFlow. Isso é evidente pelo fato de haver uma branch 'develop' detectada e a presença de branches para desenvolvimento (como 'v0.7.8' e 'docker-rebuild-v0.7.8'), branches de feature (como '0.7.3') e uma branch de produção (como'master'). O GitFlow é um modelo de branching que promove a separação clara entre branches de desenvolvimento, branches de feature e branches de produção, o que facilita a manutenção e a colaboração em projetos de longo prazo.

2. A estratégia de release do projeto Crawl4AI parece ser a Release Train. Isso é baseado no fato de haver uma sequência de versões com incrementos semânticos (como v0.7.8, v0.7.7, v0.7.6, etc.) e a presença de um Changelog que documenta as alterações de cada versão. A Release Train é uma estratégia de release que envolve a criação de novas versões com frequência regular, como por exemplo, a cada mês ou a cada três meses.

3. O fluxo de trabalho observad